# Importing Packages

In [41]:
import pandas as pd
import numpy as np
import pickle
import re

%reload_ext autoreload
%autoreload 2
import sys
sys.path.append("../py")
from utils import *
from preprocess import *
from sklearn.model_selection import train_test_split

#Binary Dataset

In [42]:
df1 = pd.read_csv("../data/dataset/binary_dataset.csv", index_col=0)


In [43]:
df1

,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm
...,...,...,...,...,...,...,...,...,...,...
40736,9995,we should forcibly remove all the foreigners f...,hate,notgiven,True,0.86945,dynabench entry,1,train,cagnluiznm
40737,9996,we should forcibly remove all the immigrants f...,hate,notgiven,True,0.50039,dynabench entry,1,train,cagnluiznm
40738,9997,dalits are unworthy of the Gods love,hate,notgiven,True,0.97652,dynabench entry,1,train,cagnluiznm
40739,9998,dalits are worthless,hate,notgiven,True,0.94326,dynabench entry,1,train,cagnluiznm


In [44]:
df1 = df1.drop(columns=['id', 'type', 'model_wrong', 'db.model_preds','status','round','split','annotator'], axis=1)
df1.columns = ['text', 'target']
df1.head()

,text,target
1,It should be legal for a husband to rape his wife,hate
2,dalits are lowlife scum,hate
3,dalits are lowlives,hate
4,It was a better world when women dare not ques...,hate
5,I don't work this hard so that those immigrant...,hate


In [45]:
df1.target.value_counts()

hate       22124
nothate    18499
Name: target, dtype: int64

In [46]:
df1['target'].unique()


array(['hate', 'nothate'], dtype=object)

In [47]:
df1['target'].replace(['nothate','hate'], [0,1], inplace=True)
df1

,text,target
1,It should be legal for a husband to rape his wife,1
2,dalits are lowlife scum,1
3,dalits are lowlives,1
4,It was a better world when women dare not ques...,1
5,I don't work this hard so that those immigrant...,1
...,...,...
40736,we should forcibly remove all the foreigners f...,1
40737,we should forcibly remove all the immigrants f...,1
40738,dalits are unworthy of the Gods love,1
40739,dalits are worthless,1


In [48]:
df1.columns=['tweet','target']

In [49]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


# 2ND DATASET

In [118]:
df2 = pd.read_csv("https://raw.githubusercontent.com/hafidhfikri/Practice-Twitter-Sentiment-Analysis/master/train_E6oV3lV.csv")
df2

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [119]:
df2=df2.drop(columns=['id'],axis=1);

In [124]:
df2.target.value_counts()

0    29720
1     2242
Name: target, dtype: int64

In [126]:
df2.columns=['target','tweet']
df2=df2[['tweet','target']]

# Balancing DataSet By OverSampling and UnderSampling

In [128]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

class_counts = df2['target'].value_counts()
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()
# Determine the class with fewer instances
minority_class_count = class_counts[minority_class]
majority_class_count = class_counts[majority_class]

#Apply oversampling to minority class
oversampler = RandomOverSampler(sampling_strategy='minority')
X = df2.drop('target', axis=1)  # Assuming the label column is named 'label'
y = df2['target']
X_oversampled, y_oversampled = oversampler.fit_resample(X, y)
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_balanced, y_balanced = undersampler.fit_resample(X_oversampled, y_oversampled)
# Verify the balanced class distribution
balanced_class_counts = pd.Series(y_balanced).value_counts()
print('Balanced Class Distribution:')
print(balanced_class_counts)
df2b = pd.concat([X_balanced,y_balanced], axis=1)


Balanced Class Distribution:
0    29720
1    29720
Name: target, dtype: int64


In [133]:
import pickle
pickle.dump(df2b, open("../pickle/balanced2.pickle", "wb"))


In [135]:
df2b

,tweet,target
0,looking forward to the @user testimonial,0
1,what in the world is donald trump saying if yo...,0
2,crystal guidance #crystaltips â #crystals fo...,0
3,loving today!!!! got a this as a present from ...,0
4,sad how muslims eat during the ramadaan.. #n...,0
...,...,...
59435,it's clear that we're having a crisis of #comp...,1
59436,@user you areclueless buddy regarding long sta...,1
59437,oops! #carlpaladino didn't mean to publicly po...,1
59438,"#hatred,#greed&amp; destroyed many lives,but m...",1


In [136]:
# df2_balanced = pickle.load(open("../pickle/balanced2.pickle", "rb"))
# df2_balanced.head()
df2_bal=df2b

#Dividing Data into Training,Validation and Testing

In [137]:
hate = df2_bal[df2_bal.target == 1]
non_hate = df2_bal[df2_bal.target == 0]

X_h = hate.tweet
y_h = hate.target
X_nh = non_hate.tweet
y_nh = non_hate.target

X_h_tr, X_h_val, y_h_tr, y_h_val = train_test_split(X_h, y_h, test_size=0.25, random_state=42)
X_h_val, X_h_tt, y_h_val, y_h_tt = train_test_split(X_h_val, y_h_val, test_size=0.4, random_state=42)
X_nh_tr, X_nh_val, y_nh_tr, y_nh_val = train_test_split(X_nh, y_nh, test_size=0.25, random_state=42)
X_nh_val, X_nh_tt, y_nh_val, y_nh_tt = train_test_split(X_nh_val, y_nh_val, test_size=0.4, random_state=42)

In [138]:
X_tr = pd.concat((X_h_tr, X_nh_tr), ignore_index=True)
y_tr = pd.concat((y_h_tr, y_nh_tr), ignore_index=True)
train_bal = pd.concat([X_tr, y_tr], axis=1)
train_bal.head()

,tweet,target
0,the latest the indigenous and remote daily! t...,1
1,@user my video on the whole @user situation #b...,1
2,trumps associates are a classy bunch.,1
3,@user @user it is people like you whom they've...,1
4,black men and black women with stockholm syndr...,1


In [139]:
X_val = pd.concat((X_h_val, X_nh_val), ignore_index=True)
y_val = pd.concat((y_h_val, y_nh_val), ignore_index=True)
val_bal = pd.concat([X_val, y_val], axis=1)
val_bal.head()
X_tt = pd.concat((X_h_tt, X_nh_tt), ignore_index=True)
y_tt = pd.concat((y_h_tt, y_nh_tt), ignore_index=True)
test_bal = pd.concat([X_tt, y_tt], axis=1)
test_bal.head()

,tweet,target
0,@user #spoty perhaps the #murrayhaters would b...,1
1,you might be a libtard if... #libtard #sjw #l...,1
2,"#koreans &amp; joseon people in japan, will ab...",1
3,@user and #ny #developer @user ð¿ racist#de...,1
4,#fascism doesn't arrive doing a #nazi salut.bu...,1


In [140]:
pickle.dump(test_bal, open("../pickle/test_bal.pickle", "wb"))
pickle.dump(val_bal, open("../pickle/val_bal.pickle", "wb"))
pickle.dump(train_bal, open("../pickle/train_bal.pickle", "wb"))

#Davidson Dataset

In [81]:
df = pd.read_csv("https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv", index_col=0)
df = df.drop(columns=['count', 'hate_speech', 'offensive_language', 'neither'], axis=1)
df.columns = ['target', 'text']
df.head()

,target,text
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [83]:
df = df.replace({'target': {1: 1, 2: 1, 0: 0}})

In [86]:
df=df[['text','target']]

In [87]:
df

,text,target
0,!!! RT @mayasolovely: As a woman you shouldn't...,1
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
25291,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
25292,"you've gone and broke the wrong heart baby, an...",1
25294,young buck wanna eat!!.. dat nigguh like I ain...,1
25295,youu got wild bitches tellin you lies,1


# Renaming columns

In [90]:
df.columns = ['tweet', 'target']

In [91]:
df.target.value_counts()

1    23353
0     1430
Name: target, dtype: int64

# Combing first and third Dataset

In [92]:
df = pd.concat([df1, df], ignore_index=True)
df

,tweet,target
0,It should be legal for a husband to rape his wife,1
1,dalits are lowlife scum,1
2,dalits are lowlives,1
3,It was a better world when women dare not ques...,1
4,I don't work this hard so that those immigrant...,1
...,...,...
65401,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
65402,"you've gone and broke the wrong heart baby, an...",1
65403,young buck wanna eat!!.. dat nigguh like I ain...,1
65404,youu got wild bitches tellin you lies,1


In [93]:
df['target'].value_counts()

1    45477
0    19929
Name: target, dtype: int64

In [94]:
class_counts = df['target'].value_counts()
minority_class = class_counts.idxmin()
majority_class = class_counts.idxmax()


In [95]:
# Determine the class with fewer instances
minority_class_count = class_counts[minority_class]
majority_class_count = class_counts[majority_class]

In [96]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [97]:
#Apply oversampling to minority class
oversampler = RandomOverSampler(sampling_strategy='minority')
X = df.drop('target', axis=1)  # Assuming the label column is named 'label'
y = df['target']
X_oversampled, y_oversampled = oversampler.fit_resample(X, y)


In [98]:
# Apply undersampling to majority class
undersampler = RandomUnderSampler(sampling_strategy='majority')
X_balanced, y_balanced = undersampler.fit_resample(X_oversampled, y_oversampled)

In [99]:
# Verify the balanced class distribution
balanced_class_counts = pd.Series(y_balanced).value_counts()
print('Balanced Class Distribution:')
print(balanced_class_counts)
dfb = pd.concat([X_balanced,y_balanced], axis=1)

Balanced Class Distribution:
0    45477
1    45477
Name: target, dtype: int64


In [101]:
dfb

,tweet,target
0,"""you tiny piece of brown crap go back to Mexic...",0
1,"let's be honest, how many of you want to becom...",0
2,"Around the world, you will find a number of pe...",0
3,Comments such as gay people are disgusting s...,0
4,anyone who says that drug dealers are trustwor...,0
...,...,...
90949,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
90950,"you've gone and broke the wrong heart baby, an...",1
90951,young buck wanna eat!!.. dat nigguh like I ain...,1
90952,youu got wild bitches tellin you lies,1


In [102]:
dfb['target'].value_counts()

0    45477
1    45477
Name: target, dtype: int64

# Pickle Out Dataframe

In [103]:
import pickle

# model_filename = 'ufo-model.pkl'
# pickle.dump(model, open(model_filename,'wb'))
pickle.dump(dfb, open("../pickle/balanced.pickle", "wb"))

# Train-Validation-Test

In [104]:
df_balanced = pickle.load(open("../pickle/balanced.pickle", "rb"))
df_balanced.head()

,tweet,target
0,"""you tiny piece of brown crap go back to Mexic...",0
1,"let's be honest, how many of you want to becom...",0
2,"Around the world, you will find a number of pe...",0
3,Comments such as gay people are disgusting s...,0
4,anyone who says that drug dealers are trustwor...,0


In [105]:
df_balanced.target.value_counts()

0    45477
1    45477
Name: target, dtype: int64

In [106]:
df_bal = df_balanced[['tweet', 'target']] 
df_bal.target.value_counts()


0    45477
1    45477
Name: target, dtype: int64

In [107]:
df_bal

,tweet,target
0,"""you tiny piece of brown crap go back to Mexic...",0
1,"let's be honest, how many of you want to becom...",0
2,"Around the world, you will find a number of pe...",0
3,Comments such as gay people are disgusting s...,0
4,anyone who says that drug dealers are trustwor...,0
...,...,...
90949,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
90950,"you've gone and broke the wrong heart baby, an...",1
90951,young buck wanna eat!!.. dat nigguh like I ain...,1
90952,youu got wild bitches tellin you lies,1


In [108]:
hate = df_bal[df_bal.target == 1]
non_hate = df_bal[df_bal.target == 0]

X_h = hate.tweet
y_h = hate.target
X_nh = non_hate.tweet
y_nh = non_hate.target

X_h_tr, X_h_val, y_h_tr, y_h_val = train_test_split(X_h, y_h, test_size=0.25, random_state=42)
X_h_val, X_h_tt, y_h_val, y_h_tt = train_test_split(X_h_val, y_h_val, test_size=0.4, random_state=42)
X_nh_tr, X_nh_val, y_nh_tr, y_nh_val = train_test_split(X_nh, y_nh, test_size=0.25, random_state=42)
X_nh_val, X_nh_tt, y_nh_val, y_nh_tt = train_test_split(X_nh_val, y_nh_val, test_size=0.4, random_state=42)

In [109]:
X_tr = pd.concat((X_h_tr, X_nh_tr), ignore_index=True)
y_tr = pd.concat((y_h_tr, y_nh_tr), ignore_index=True)
train_bal = pd.concat([X_tr, y_tr], axis=1)
train_bal.head()

,tweet,target
0,I tell people that I do not think brown people...,1
1,I would ot like to have a troon as a neighbour,1
2,@BjwalkinRIVERS bro that hoe live,1
3,So apparently me &amp; Eric fucced &amp; I be ...,1
4,I hate it when birds want you to pay for every...,1


In [110]:
df_bal[df_bal.target == 0]

,tweet,target
0,"""you tiny piece of brown crap go back to Mexic...",0
1,"let's be honest, how many of you want to becom...",0
2,"Around the world, you will find a number of pe...",0
3,Comments such as gay people are disgusting s...,0
4,anyone who says that drug dealers are trustwor...,0
...,...,...
45472,there is so much to be proud of if you are bri...,0
45473,I would love to see more stores open and not c...,0
45474,@asshole_king pats owner is a kike.,0
45475,There's another concern that has been put to t...,0


In [111]:
X_val = pd.concat((X_h_val, X_nh_val), ignore_index=True)
y_val = pd.concat((y_h_val, y_nh_val), ignore_index=True)
val_bal = pd.concat([X_val, y_val], axis=1)
val_bal.head()


,tweet,target
0,boomers are the dumbest generation ever,1
1,My street is packed with Diaper-Heads,1
2,"If I hold the door open for you, bitch you BET...",1
3,"Bro's before hoes, not bro's over your girlfri...",1
4,"I'm not your slave, asshole. You cotton picker...",1


In [112]:
X_tt = pd.concat((X_h_tt, X_nh_tt), ignore_index=True)
y_tt = pd.concat((y_h_tt, y_nh_tt), ignore_index=True)
test_bal = pd.concat([X_tt, y_tt], axis=1)
test_bal.head()

,tweet,target
0,"Notice I didn't tweet ""bytches"".... Bytch is d...",1
1,"I would hate to be called a chink, lucky I am ...",1
2,IMAGINE A WORLD WITHOUT POOFS.. now let's make...,1
3,Thinking bout gettin 500$ put on my ears cuz I...,1
4,"""I'm probably your main bitch chocolate dipped...",1


In [115]:
train_bal.target.value_counts()

1    34107
0    34107
Name: target, dtype: int64

In [116]:
pickle.dump(test_bal, open("../pickle/test_bal.pickle", "wb"))
pickle.dump(val_bal, open("../pickle/val_bal.pickle", "wb"))
pickle.dump(train_bal, open("../pickle/train_bal.pickle", "wb"))